# Combination Sampling

Implement the SMOTEENN technique with the credit card default data. Then estimate a logistic regression model and report the classification evaluation metrics.

ln_balance_limit is the log of the maximum balance they can have on the card; 1 is female, 0 male for sex; the education is denoted: 1 = graduate school; 2 = university; 3 = high school; 4 = others; 1 is married and 0 single for marriage; default_next_month is whether the person defaults in the following month (1 yes, 0 no).

In [1]:
import pandas as pd
from path import Path
from collections import Counter

In [2]:
data = Path('../Resources/cc_default.csv')
df = pd.read_csv(data)

In [3]:
x_cols = [i for i in df.columns if i not in ('ID', 'default_next_month')]
X = df[x_cols]
y = df['default_next_month']

In [4]:
x_cols

['ln_balance_limit', 'sex', 'education', 'marriage', 'age']

In [5]:
Counter(y)

Counter({1: 6636, 0: 23364})

In [6]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Combination Sampling with SMOTEENN

In [7]:
# Use the SMOTEENN technique to perform combination sampling on the data
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Using TensorFlow backend.


Counter({0: 10151, 1: 7674})

# Logistic Regression

In [8]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Evaluation Metrics

In [9]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[4173, 1659],
       [ 876,  792]])

In [10]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.5951775616543802

In [11]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.72      0.47      0.77      0.58      0.35      5832
          1       0.32      0.47      0.72      0.38      0.58      0.33      1668

avg / total       0.71      0.66      0.53      0.68      0.58      0.34      7500



## 17.10.3
### Combination Sampling With SMOTEENN

You discuss the results of oversampling and undersampling with Jill. When you point out to her that the improvements seem to be modest, she explains that incremental improvements are usually more realistic than drastic ones. Jill also tells you that such small improvements, in tandem with other tweaks, can add up to make a significant difference. For now, however, she suggests learning about SMOTEENN, an approach to resampling that combines aspects of both oversampling and undersampling.

####

As previously discussed, a downside of oversampling with SMOTE is its reliance on the immediate neighbors of a data point. Because the algorithm doesn't see the overall distribution of data, the new data points it creates can be heavily influenced by outliers. This can lead to noisy data. With downsampling, the downsides are that it involves loss of data and is not an option when the dataset is small. One way to deal with these challenges is to use a sampling strategy that is a combination of oversampling and undersampling.

SMOTEENN combines the SMOTE and Edited Nearest Neighbors (ENN) algorithms. SMOTEENN is a two-step process:

   1. Oversample the minority class with SMOTE.
   2. Clean the resulting data with an undersampling strategy. If the two nearest neighbors of a data point belong to two different classes, that data point is dropped.

The series of images below help illustrate the SMOTEENN technique. The first image represents a synthetically generated dataset (using the make_blobs module) and shows two classes: purple as the majority class and yellow as the minority class.

Visualization of a dataset with class imbalance.

In the following image, the minority class is oversampled with SMOTE.

data-17-10-3-2-Oversampling-Smote.png

Note that the two classes significantly overlap, as the box indicates below. This overlap makes classification difficult.

Oversampling with SMOTE can create overlapping classes.

In the next image, SMOTEENN is applied, instead of SMOTE. As with SMOTE, the minority class is oversampled; however, an undersampling step is added, removing some of each class's outliers from the dataset. The result is that the two classes are separated more cleanly.

SMOTEENN removes outliers from the dataset after oversampling.

Let's apply SMOTEENN to the credit card default dataset and compare its results.

Download 17-10-3-combination_sampling.zip (Links to an external site.) **THIS IS THE ABOVE CODE**

The code is much the same as before. The only difference is in using the SMOTEENN module.

    import pandas as pd
    from path import Path
    from collections import Counter

    data = Path('../Resources/cc_default.csv')
    df = pd.read_csv(data)
    df.head()

Peview of the dataset.

Again, the ID and default_next_month columns are filtered to create the features dataset. The default_next_month column is defined as the target dataset.

    x_cols = [i for i in df.columns if i not in ('ID', 'default_next_month')]
    X = df[x_cols]
    y = df['default_next_month']

The dataset is split into training and testing sets.

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Next, we import the SMOTEENN module and create an instance of SMOTEENN, which resamples the dataset.

    from imblearn.combine import SMOTEENN
    smote_enn = SMOTEENN(random_state=0)
    X_resampled, y_resampled = smote_enn.fit_resample(X, y)

Again, we use a LogisticRegression model to generate predictions.

    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_resampled, y_resampled)

As before, the evaluation metrics are generated. First, the confusion_matrix is generated.

    from sklearn.metrics import confusion_matrix
    y_pred = model.predict(X_test)
    confusion_matrix(y_test, y_pred)

Next, the balanced_accuracy_score is generated.

    from sklearn.metrics import balanced_accuracy_score
    balanced_accuracy_score(y_test, y_pred)

Finally, we print the classification report.

    from imblearn.metrics import classification_report_imbalanced
    print(classification_report_imbalanced(y_test, y_pred))

The classification report calculates the precision, recall (sensitivity), and F1 score.

Resampling with SMOTEENN did not work miracles, but some of the metrics show an improvement over undersampling.